In [10]:
import numpy as np
import pandas as pd
import scipy as sp
import os
import string
import glob
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

# 1. Wouter style preprocessing

In [2]:
files = glob.glob('../data/txt/*.txt')

In [8]:
# list of lists, so we can keep info about spelling variants in individual arrays

stops = []

with open('../data/functionwords.txt','r') as file:
    for line in file:
        words = [x for x in line.lower().split() if x != '/']
        stops.append(words)
            
print(stops[:3])

[['atqui', 'atqvi'], ['autem', 'avtem'], ['certe']]


In [56]:
folder_path = '../data/txt'

def preprocess_text(text):
    # lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    return text

def slice_text_into_samples(text, sample_size=1000):
    text = text.split()
    num_samples = len(text) // sample_size
    samples = [text[i * sample_size : (i + 1) * sample_size] for i in range(num_samples)]
    
    return samples

def preprocess_and_slice_text_files(folder_path, sample_size=1000):
    processed_texts = {}
    names = []
    translators = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") and not filename.startswith("Myst"): # i'm skipping the Mysery files
            with open(os.path.join(folder_path, filename), "r", encoding='latin-1') as file:
                text = file.read()
                processed_text = preprocess_text(text)
                samples = slice_text_into_samples(processed_text, sample_size)
                num_samples = len(samples)
                for i, sample in enumerate(samples):
                    variable_name = filename[:-4] + "_" + str(i)
                    processed_texts[variable_name] = sample
            print(f"'{filename}' was processed and split into {num_samples} samples")
    return processed_texts

In [87]:
preprocess = preprocess_and_slice_text_files(folder_path)

'BurCom.txt' was processed and split into 24 samples
'AristPha.txt' was processed and split into 20 samples
'BurInt.txt' was processed and split into 64 samples
'AnonMetaph.txt' was processed and split into 61 samples
'BurEthN.txt' was processed and split into 4 samples
'BarMor.txt' was processed and split into 20 samples
'AristMet.txt' was processed and split into 6 samples
'BarMirab.txt' was processed and split into 7 samples
'WilMet.txt' was processed and split into 30 samples
'JamPhys.txt' was processed and split into 53 samples
'WilInPar.txt' was processed and split into 161 samples
'BarMun.txt' was processed and split into 5 samples
'WilTet.txt' was processed and split into 33 samples
'BarSig.txt' was processed and split into 3 samples
'AristPhaP.txt' was processed and split into 0 samples
'BarPri.txt' was processed and split into 2 samples
'WilInTim.txt' was processed and split into 4 samples
'JamMetaph.txt' was processed and split into 15 samples
'BurNemP.txt' was processed and

# 2. Chunk dataframe

In [92]:
entries = []
for k, txt in preprocess.items():
    ww = re.findall('[A-Z][a-z]*', k)
    transl = ww[0]
    work = ''.join(ww[1:])
    chunk = ' '.join(txt)
    entries.append({
        'Translator' : transl,
        'Work' : work,
        'Chunk' : chunk
        })
chunk_df = pd.DataFrame(entries)
chunk_df

,Translator,Work,Chunk
0,Bur,Com,quoniam quidem ex calido et frigido et sicco e...
1,Bur,Com,sermo non quod nunquam fit in uno eodemque cor...
2,Bur,Com,horis anni invenire quartam coniugationem comp...
3,Bur,Com,quod necesse est in ea putrefieri omnia incipi...
4,Bur,Com,de ipsis per capitula quantum ad presentia uti...
...,...,...,...
1152,Bur,Fid,ostendat quod secundum veritatem est homo cum ...
1153,Bur,Fid,causative dicere ut hoc tibi soli peccavi et p...
1154,Bur,Fid,condemnavit peccatum in carne ut iustitia legi...
1155,Bur,Fid,uxorem suam et concepit et genuit quare propte...


In [93]:
set(chunk_df.Work)

{'Alex',
 'Anim',
 'Cael',
 'Com',
 'Elem',
 'EthN',
 'EthV',
 'Fid',
 'Gen',
 'GenA',
 'His',
 'Hom',
 'InPar',
 'InTim',
 'Int',
 'Mat',
 'Men',
 'Met',
 'Metaph',
 'Mirab',
 'Mor',
 'Mun',
 'P',
 'Pha',
 'Phys',
 'Pri',
 'Pue',
 'Sig',
 'Simp',
 'Tet',
 'X'}

# 3. Vectorize, select only keyword freqs, merge spelling variants

In [94]:
vectorizer = TfidfVectorizer(use_idf=False, analyzer='word', decode_error='replace')
X = vectorizer.fit_transform(chunk_df.Chunk)


In [95]:
# There is some encoding horror happening here, but the Latin stops won't be affected (hopefully?!)

df = pd.DataFrame(X.toarray())
df.columns = vectorizer.get_feature_names_out()
df.head()

,10,130,1capite,326b29,327a30,426b,aa,aaa,aaron,ab,...,î½îµá,î½îµî,î½îµîºîµî½î,î½îµï,î½î½á,î½ï,î¾á½,î¾îµî¹ï,î¾îµï,ügnotum
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017303,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063885,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008073,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
flat_stops = [item for sublist in stops for item in sublist]

stops_df = df[df.columns.intersection(flat_stops)]

In [97]:
# Just our Pieter stopwords now, but not all of the (eg no 'atqvi')

stops_df

,atqui,autem,avtem,certe,ceu,confestim,cum,cvm,dehinc,deinceps,...,uidelicet,ut,utique,velut,veluti,velvt,vero,videlicet,vt,vtiqve
0,0.0,0.190328,0.0,0.0,0.0,0.0,0.051908,0.0,0.0,0.008651,...,0.0,0.051908,0.077862,0.008651,0.0,0.0,0.077862,0.0,0.0,0.0
1,0.0,0.229559,0.0,0.0,0.0,0.0,0.061804,0.0,0.0,0.000000,...,0.0,0.088292,0.123609,0.008829,0.0,0.0,0.026488,0.0,0.0,0.0
2,0.0,0.228161,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.009126,...,0.0,0.100391,0.054759,0.018253,0.0,0.0,0.091264,0.0,0.0,0.0
3,0.0,0.333577,0.0,0.0,0.0,0.0,0.017557,0.0,0.0,0.000000,...,0.0,0.131675,0.079005,0.000000,0.0,0.0,0.017557,0.0,0.0,0.0
4,0.0,0.234115,0.0,0.0,0.0,0.0,0.040365,0.0,0.0,0.008073,...,0.0,0.226042,0.121094,0.000000,0.0,0.0,0.040365,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,0.0,0.191573,0.0,0.0,0.0,0.0,0.060955,0.0,0.0,0.000000,...,0.0,0.208988,0.008708,0.000000,0.0,0.0,0.008708,0.0,0.0,0.0
1153,0.0,0.162756,0.0,0.0,0.0,0.0,0.019148,0.0,0.0,0.000000,...,0.0,0.086165,0.019148,0.009574,0.0,0.0,0.028722,0.0,0.0,0.0
1154,0.0,0.229977,0.0,0.0,0.0,0.0,0.029997,0.0,0.0,0.000000,...,0.0,0.149985,0.019998,0.009999,0.0,0.0,0.009999,0.0,0.0,0.0
1155,0.0,0.170615,0.0,0.0,0.0,0.0,0.040145,0.0,0.0,0.000000,...,0.0,0.070253,0.010036,0.000000,0.0,0.0,0.010036,0.0,0.0,0.0


In [98]:
# Now sum columns for all spelling variants, using the first word as the new column name

stops_df_combined = pd.DataFrame()
for ary in stops:
    stops_df_combined[ary[0]] = stops_df[stops_df.columns.intersection(ary)].sum(1)

In [99]:
stops_df_combined

,atqui,autem,certe,ceu,confestim,cum,dehinc,deinceps,demum,denique,...,sepe,sicut,siquidem,tamquam,ut,utique,velut,veluti,vero,videlicet
0,0.0,0.190328,0.0,0.0,0.0,0.051908,0.0,0.008651,0.0,0.008651,...,0.0,0.000000,0.000000,0.0,0.051908,0.077862,0.008651,0.0,0.077862,0.0
1,0.0,0.229559,0.0,0.0,0.0,0.061804,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.008829,0.0,0.088292,0.123609,0.008829,0.0,0.026488,0.0
2,0.0,0.228161,0.0,0.0,0.0,0.000000,0.0,0.009126,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.100391,0.054759,0.018253,0.0,0.091264,0.0
3,0.0,0.333577,0.0,0.0,0.0,0.017557,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.131675,0.079005,0.000000,0.0,0.017557,0.0
4,0.0,0.234115,0.0,0.0,0.0,0.040365,0.0,0.008073,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.226042,0.121094,0.000000,0.0,0.040365,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,0.0,0.191573,0.0,0.0,0.0,0.060955,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.208988,0.008708,0.000000,0.0,0.008708,0.0
1153,0.0,0.162756,0.0,0.0,0.0,0.019148,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.086165,0.019148,0.009574,0.0,0.028722,0.0
1154,0.0,0.229977,0.0,0.0,0.0,0.029997,0.0,0.000000,0.0,0.009999,...,0.0,0.009999,0.000000,0.0,0.149985,0.019998,0.009999,0.0,0.009999,0.0
1155,0.0,0.170615,0.0,0.0,0.0,0.040145,0.0,0.000000,0.0,0.000000,...,0.0,0.020072,0.000000,0.0,0.070253,0.010036,0.000000,0.0,0.010036,0.0


In [100]:
# finally, merge in the metadata columns (they can be dropped out later to make X and y arrays)

stops_tidy = pd.concat([chunk_df,stops_df_combined],axis=1)
stops_tidy

,Translator,Work,Chunk,atqui,autem,certe,ceu,confestim,cum,dehinc,...,sepe,sicut,siquidem,tamquam,ut,utique,velut,veluti,vero,videlicet
0,Bur,Com,quoniam quidem ex calido et frigido et sicco e...,0.0,0.190328,0.0,0.0,0.0,0.051908,0.0,...,0.0,0.000000,0.000000,0.0,0.051908,0.077862,0.008651,0.0,0.077862,0.0
1,Bur,Com,sermo non quod nunquam fit in uno eodemque cor...,0.0,0.229559,0.0,0.0,0.0,0.061804,0.0,...,0.0,0.000000,0.008829,0.0,0.088292,0.123609,0.008829,0.0,0.026488,0.0
2,Bur,Com,horis anni invenire quartam coniugationem comp...,0.0,0.228161,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.100391,0.054759,0.018253,0.0,0.091264,0.0
3,Bur,Com,quod necesse est in ea putrefieri omnia incipi...,0.0,0.333577,0.0,0.0,0.0,0.017557,0.0,...,0.0,0.000000,0.000000,0.0,0.131675,0.079005,0.000000,0.0,0.017557,0.0
4,Bur,Com,de ipsis per capitula quantum ad presentia uti...,0.0,0.234115,0.0,0.0,0.0,0.040365,0.0,...,0.0,0.000000,0.000000,0.0,0.226042,0.121094,0.000000,0.0,0.040365,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,Bur,Fid,ostendat quod secundum veritatem est homo cum ...,0.0,0.191573,0.0,0.0,0.0,0.060955,0.0,...,0.0,0.000000,0.000000,0.0,0.208988,0.008708,0.000000,0.0,0.008708,0.0
1153,Bur,Fid,causative dicere ut hoc tibi soli peccavi et p...,0.0,0.162756,0.0,0.0,0.0,0.019148,0.0,...,0.0,0.000000,0.000000,0.0,0.086165,0.019148,0.009574,0.0,0.028722,0.0
1154,Bur,Fid,condemnavit peccatum in carne ut iustitia legi...,0.0,0.229977,0.0,0.0,0.0,0.029997,0.0,...,0.0,0.009999,0.000000,0.0,0.149985,0.019998,0.009999,0.0,0.009999,0.0
1155,Bur,Fid,uxorem suam et concepit et genuit quare propte...,0.0,0.170615,0.0,0.0,0.0,0.040145,0.0,...,0.0,0.020072,0.000000,0.0,0.070253,0.010036,0.000000,0.0,0.010036,0.0


In [101]:
stops_tidy.to_csv('../data/stops_tidy.csv')